### Setup

In [ ]:
from __future__ import print_function

import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf

import sys
sys.path.append('..')
import models.VAE as vae

import os
from io import BytesIO
import PIL.Image
import scipy.misc
import scipy.io
from IPython.display import clear_output, Image, display

np.random.seed(0)

In [ ]:
# load data
import models.DataReader as Data

# get data handler
data_type = 'mnist'
data_dir = '/home/mattw/Dropbox/git/dreamscape/data/'

if data_type is 'mnist':
    data = Data.DataReaderMNIST(data_dir + 'mnist/', one_hot=True)
elif data_type is 'cifar':
    data = Data.DataReaderCIFAR(data_dir + 'cifar/', one_hot=True)  

### GIF animation of latent space during training

In [ ]:
reload(vae)

# store processed images
saving = True
save_path = '/home/mattw/Desktop/test_movie'  

# define model params
layers_encoder = [784, 400, 400]
layer_latent = 2
layers_decoder = [400, 400, 784]

# initialize model
net = vae.VAE(
    layers_encoder=layers_encoder, 
    layer_latent=layer_latent,
    layers_decoder=layers_decoder)

# define training params
batch_size = 128
use_gpu = 1
iters_per_image = np.logspace(0, 3, num=50).astype(int)
iters_disp = None
iters_ckpt = None
iters_summary = None

# image details
nx = ny = 20
x_values = np.linspace(-3, 3, nx)
y_values = np.linspace(-3, 3, ny)

# start the tensorflow session
config = tf.ConfigProto(device_count = {'GPU': use_gpu})
sess = tf.Session(config=config, graph=net.graph)
sess.run(net.init)

for epoch, iters_training in enumerate(iters_per_image):
    
    # output updates
    print('\rEpoch %03g of %03g - training for %05g iters' % 
          (epoch + 1, iters_per_image.size, iters_per_image[epoch]), end='')
    
    # train model
    net.train_iters(
        sess, 
        data=data,
        batch_size=batch_size,
        iters_training=iters_training,
        iters_disp=iters_disp)
    
    # create latent state representation
    canvas = np.empty((28*ny, 28*nx))
    for i, yi in enumerate(x_values):
        for j, xi in enumerate(y_values):
            z_mean = np.array([[xi, yi]])
            x_mean = net.generate(sess, z_mean=z_mean)
            canvas[(nx-i-1)*28:(nx-i)*28, j*28:(j+1)*28] = x_mean[0].reshape(28, 28)
    
    # save output
    if saving:
        file_name = str('vae-mnist_epoch_%03i.jpg' % epoch)
        file_path = '/'.join([save_path, file_name])
        canvas = np.uint8(255*canvas)
        PIL.Image.fromarray(canvas).save(file_path, 'jpeg')
        
sess.close()

### to create a gif:
convert -delay 4x120 -loop 0 *.jpg animated.gif

take all jpgs in current directory and turn them into a gif that loops indefinitely, with a framerate of 120/4 = 30 fps

In [1]:
%reload_ext watermark
%watermark -a "Matt Whiteway" -d -v -m -p numpy,tensorflow

Matt Whiteway 2017-06-13 

CPython 2.7.12
IPython 5.3.0

numpy 1.11.0
tensorflow 1.0.1

compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.4.0-78-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit
